# 이미지 타일 인식 (CAPTCHA-like)
## 1x1 비율 이미지를 9개 타일로 분할하여 특정 객체 인식

---

### 프로젝트 개요
- **목적**: CAPTCHA처럼 이미지를 9개(3x3) 타일로 나누고, 특정 객체가 포함된 타일을 예측
- **기술**: CNN (Convolutional Neural Network) 딥러닝
- **환경**: Google Colab

## 1. 라이브러리 임포트

In [ ]:
# 기본 라이브러리
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 이미지 처리
from PIL import Image
import cv2

# 딥러닝 라이브러리 (TensorFlow & Keras)
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# 머신러닝 유틸리티
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix

# Google Colab 파일 업로드
from google.colab import files

# 경고 무시
import warnings
warnings.filterwarnings('ignore')

print("라이브러리 임포트 완료!")
print(f"TensorFlow 버전: {tf.__version__}")

## 2. 이미지 파일 업로드

In [ ]:
# PNG 이미지 파일 업로드
print("PNG 이미지 파일을 업로드해주세요:")
uploaded = files.upload()

# 업로드된 파일 목록
uploaded_files = list(uploaded.keys())
print(f"\n업로드된 파일: {uploaded_files}")

## 3. 이미지 전처리 함수

In [ ]:
def load_and_resize_image(image_path, target_size=300):
    """
    이미지를 로드하고 1:1 비율로 리사이즈
    
    Parameters:
    - image_path: 이미지 파일 경로
    - target_size: 리사이즈 크기 (정사각형)
    
    Returns:
    - 리사이즈된 이미지 배열
    """
    img = Image.open(image_path)
    img = img.convert('RGB')  # RGB로 변환
    img = img.resize((target_size, target_size))  # 1:1 비율로 리사이즈
    img_array = np.array(img) / 255.0  # 정규화 (0~1 범위)
    return img_array


def split_image_into_tiles(image_array, num_tiles=3):
    """
    이미지를 num_tiles x num_tiles 타일로 분할 (기본 3x3 = 9개)
    
    Parameters:
    - image_array: 이미지 배열 (H, W, C)
    - num_tiles: 한 축당 타일 개수 (기본 3)
    
    Returns:
    - tiles: 분할된 타일 리스트 [(타일0, 위치0), (타일1, 위치1), ...]
    """
    height, width, channels = image_array.shape
    tile_height = height // num_tiles
    tile_width = width // num_tiles
    
    tiles = []
    tile_positions = []
    
    for i in range(num_tiles):
        for j in range(num_tiles):
            # 타일 추출
            tile = image_array[
                i * tile_height:(i + 1) * tile_height,
                j * tile_width:(j + 1) * tile_width,
                :
            ]
            tiles.append(tile)
            tile_positions.append((i, j))  # 타일 위치 저장 (행, 열)
    
    return tiles, tile_positions


def visualize_tiles(tiles, tile_positions, figsize=(12, 12)):
    """
    9개 타일을 3x3 그리드로 시각화
    """
    fig, axes = plt.subplots(3, 3, figsize=figsize)
    
    for idx, (tile, pos) in enumerate(zip(tiles, tile_positions)):
        row, col = pos
        axes[row, col].imshow(tile)
        axes[row, col].set_title(f'타일 {idx} (위치: {pos})', fontsize=10)
        axes[row, col].axis('off')
    
    plt.tight_layout()
    plt.show()

print("이미지 전처리 함수 정의 완료!")

## 4. 이미지 로드 및 타일 분할

In [ ]:
# 첫 번째 업로드된 이미지 사용
if uploaded_files:
    sample_image_path = uploaded_files[0]
    
    # 이미지 로드 및 리사이즈
    image = load_and_resize_image(sample_image_path, target_size=300)
    print(f"이미지 shape: {image.shape}")
    
    # 원본 이미지 출력
    plt.figure(figsize=(6, 6))
    plt.imshow(image)
    plt.title('원본 이미지 (1:1 비율)', fontsize=14)
    plt.axis('off')
    plt.show()
    
    # 9개 타일로 분할
    tiles, tile_positions = split_image_into_tiles(image, num_tiles=3)
    print(f"\n분할된 타일 개수: {len(tiles)}")
    print(f"각 타일 shape: {tiles[0].shape}")
    
    # 타일 시각화
    visualize_tiles(tiles, tile_positions)
else:
    print("업로드된 이미지가 없습니다.")

## 5. CNN 모델 구축 (타일 분류용)

In [ ]:
def create_tile_classifier(input_shape=(100, 100, 3), num_classes=2):
    """
    타일 분류를 위한 CNN 모델 생성
    
    Parameters:
    - input_shape: 입력 타일 크기 (height, width, channels)
    - num_classes: 분류 클래스 수 (기본 2: 객체 있음/없음)
    
    Returns:
    - model: 컴파일된 CNN 모델
    """
    model = Sequential([
        # 첫 번째 Conv 레이어
        Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        MaxPooling2D((2, 2)),
        
        # 두 번째 Conv 레이어
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        
        # 세 번째 Conv 레이어
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        
        # Flatten 및 Fully Connected 레이어
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),  # 과적합 방지
        Dense(64, activation='relu'),
        Dropout(0.3),
        
        # 출력 레이어
        Dense(num_classes, activation='softmax')  # 이진 분류: sigmoid도 가능
    ])
    
    # 모델 컴파일
    model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',  # 이진 분류는 'binary_crossentropy'
        metrics=['accuracy']
    )
    
    return model


# 모델 생성 및 구조 확인
model = create_tile_classifier(input_shape=(100, 100, 3), num_classes=2)
model.summary()

## 6. 데이터 준비 (예시 - 실제 사용 시 수정 필요)

In [ ]:
# 데이터 생성 함수 (실제로는 라벨링된 데이터셋 필요)
def create_sample_dataset(num_samples=100, tile_size=100):
    """
    샘플 데이터셋 생성 (실제로는 라벨링된 이미지 필요)
    
    Parameters:
    - num_samples: 샘플 개수
    - tile_size: 타일 크기
    
    Returns:
    - X: 타일 이미지 배열
    - y: 라벨 배열 (0: 객체 없음, 1: 객체 있음)
    """
    # 더미 데이터 생성
    X = np.random.rand(num_samples, tile_size, tile_size, 3)
    y = np.random.randint(0, 2, size=(num_samples,))  # 0 또는 1
    
    return X, y


# 샘플 데이터 생성
X_data, y_data = create_sample_dataset(num_samples=500, tile_size=100)

print(f"X shape: {X_data.shape}")
print(f"y shape: {y_data.shape}")
print(f"\ny 클래스 분포:")
print(pd.Series(y_data).value_counts())

## 7. 학습 데이터 분할 (Train/Test Split)

In [ ]:
# One-hot 인코딩 (클래스 라벨)
from tensorflow.keras.utils import to_categorical

y_categorical = to_categorical(y_data, num_classes=2)

# Train/Test Split (80:20)
X_train, X_test, y_train, y_test = train_test_split(
    X_data, 
    y_categorical, 
    test_size=0.2, 
    random_state=42
)

print(f"학습 데이터: {X_train.shape}")
print(f"테스트 데이터: {X_test.shape}")
print(f"학습 라벨: {y_train.shape}")
print(f"테스트 라벨: {y_test.shape}")

## 8. 모델 학습 (Training)

In [ ]:
# 모델 학습
history = model.fit(
    X_train, 
    y_train,
    epochs=20,
    batch_size=32,
    validation_split=0.2,  # 검증 데이터 20%
    verbose=1
)

print("\n모델 학습 완료!")

## 9. 학습 과정 시각화

In [ ]:
# 정확도 및 손실 그래프
fig, axes = plt.subplots(ncols=2, figsize=(14, 5))

# 정확도 그래프
axes[0].plot(history.history['accuracy'], label='Train Accuracy')
axes[0].plot(history.history['val_accuracy'], label='Validation Accuracy')
axes[0].set_title('모델 정확도', fontsize=14)
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Accuracy')
axes[0].legend()
axes[0].grid(True)

# 손실 그래프
axes[1].plot(history.history['loss'], label='Train Loss')
axes[1].plot(history.history['val_loss'], label='Validation Loss')
axes[1].set_title('모델 손실', fontsize=14)
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Loss')
axes[1].legend()
axes[1].grid(True)

plt.tight_layout()
plt.show()

## 10. 모델 평가 (Evaluation)

In [ ]:
# 테스트 데이터로 예측
y_pred_proba = model.predict(X_test)
y_pred = np.argmax(y_pred_proba, axis=1)  # 가장 높은 확률의 클래스
y_test_labels = np.argmax(y_test, axis=1)

# 정확도 계산
accuracy = accuracy_score(y_test_labels, y_pred)
print(f"테스트 정확도: {accuracy:.4f} ({accuracy*100:.2f}%)")

# Confusion Matrix
cm = confusion_matrix(y_test_labels, y_pred)
print("\nConfusion Matrix:")
print(cm)

# Confusion Matrix 시각화
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['객체 없음', '객체 있음'],
            yticklabels=['객체 없음', '객체 있음'])
plt.title('Confusion Matrix', fontsize=14)
plt.ylabel('실제 값')
plt.xlabel('예측 값')
plt.show()

## 11. 타일 예측 함수 (실전 사용)

In [ ]:
def predict_tiles_with_object(image_path, model, target_size=300, tile_size=100, threshold=0.5):
    """
    이미지에서 특정 객체가 포함된 타일을 예측
    
    Parameters:
    - image_path: 이미지 파일 경로
    - model: 학습된 CNN 모델
    - target_size: 이미지 리사이즈 크기
    - tile_size: 각 타일 크기
    - threshold: 객체 포함 판단 임계값
    
    Returns:
    - selected_tiles: 객체가 포함된 타일 번호 리스트
    """
    # 이미지 로드 및 분할
    image = load_and_resize_image(image_path, target_size=target_size)
    tiles, tile_positions = split_image_into_tiles(image, num_tiles=3)
    
    # 각 타일 리사이즈 (모델 입력 크기에 맞춤)
    resized_tiles = []
    for tile in tiles:
        tile_resized = cv2.resize(tile, (tile_size, tile_size))
        resized_tiles.append(tile_resized)
    
    resized_tiles = np.array(resized_tiles)
    
    # 예측
    predictions = model.predict(resized_tiles)
    
    # 객체가 포함된 타일 선택 (클래스 1의 확률이 threshold 이상)
    selected_tiles = []
    
    print("\n=== 타일별 예측 결과 ===")
    for idx, (pred, pos) in enumerate(zip(predictions, tile_positions)):
        prob_no_object = pred[0]  # 클래스 0 (객체 없음) 확률
        prob_object = pred[1]     # 클래스 1 (객체 있음) 확률
        
        print(f"타일 {idx} (위치: {pos}): 객체 없음 {prob_no_object:.2%} | 객체 있음 {prob_object:.2%}")
        
        if prob_object >= threshold:
            selected_tiles.append(idx)
    
    print(f"\n객체가 포함된 타일: {selected_tiles}")
    
    # 시각화
    visualize_predictions(tiles, tile_positions, predictions, threshold)
    
    return selected_tiles


def visualize_predictions(tiles, tile_positions, predictions, threshold=0.5):
    """
    예측 결과를 시각화 (객체 포함 타일은 녹색 테두리)
    """
    fig, axes = plt.subplots(3, 3, figsize=(12, 12))
    
    for idx, (tile, pos, pred) in enumerate(zip(tiles, tile_positions, predictions)):
        row, col = pos
        prob_object = pred[1]  # 객체 있음 확률
        
        axes[row, col].imshow(tile)
        
        # 객체 포함 여부에 따라 제목 색상 변경
        if prob_object >= threshold:
            title_color = 'green'
            title = f'타일 {idx}\n✓ 객체 있음 ({prob_object:.2%})'
        else:
            title_color = 'red'
            title = f'타일 {idx}\n✗ 객체 없음 ({pred[0]:.2%})'
        
        axes[row, col].set_title(title, fontsize=10, color=title_color, weight='bold')
        axes[row, col].axis('off')
    
    plt.tight_layout()
    plt.show()

print("예측 함수 정의 완료!")

## 12. 새로운 이미지로 예측 테스트

In [ ]:
# 업로드된 이미지로 예측
if uploaded_files:
    test_image_path = uploaded_files[0]
    
    # 예측 실행
    selected_tiles = predict_tiles_with_object(
        image_path=test_image_path,
        model=model,
        target_size=300,
        tile_size=100,
        threshold=0.5  # 50% 이상이면 객체 포함으로 판단
    )
    
    print(f"\n최종 답변: {selected_tiles} 번 타일에 객체가 포함되어 있습니다.")
else:
    print("테스트할 이미지를 업로드해주세요.")

## 13. 모델 저장 및 로드

In [ ]:
# 모델 저장
model.save('tile_classifier_model.h5')
print("모델 저장 완료: tile_classifier_model.h5")

# 모델 로드 (필요 시)
# from tensorflow.keras.models import load_model
# loaded_model = load_model('tile_classifier_model.h5')
# print("모델 로드 완료!")

## 14. 실전 사용 가이드

### 실제 CAPTCHA 시스템 구축 방법:

1. **데이터 수집**:
   - 특정 객체(예: 신호등, 자동차, 횡단보도 등)가 포함된 이미지 수집
   - 각 이미지를 9개 타일로 분할
   - 각 타일에 라벨링 (0: 객체 없음, 1: 객체 있음)

2. **데이터 증강 (Data Augmentation)**:
   ```python
   datagen = ImageDataGenerator(
       rotation_range=20,
       width_shift_range=0.2,
       height_shift_range=0.2,
       horizontal_flip=True,
       zoom_range=0.2
   )
   ```

3. **모델 개선**:
   - Transfer Learning (VGG16, ResNet50 등 사전 학습 모델 활용)
   - Hyperparameter Tuning (학습률, 배치 크기, epoch 수 조정)
   - Ensemble 기법

4. **배포**:
   - Flask/FastAPI로 웹 API 구축
   - TensorFlow.js로 브라우저에서 직접 실행
   - TensorFlow Lite로 모바일 앱 배포

---

### 참고사항:
- 현재 코드는 **더미 데이터**로 학습됨
- 실제 사용을 위해서는 **라벨링된 데이터셋** 필요
- 데이터셋 예시: COCO, ImageNet, 직접 수집한 데이터
- GPU 사용 시 학습 속도 대폭 향상 (Colab에서 런타임 > 런타임 유형 변경 > GPU 선택)

## 15. 추가: Transfer Learning 예시 (VGG16)

In [ ]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, GlobalAveragePooling2D

def create_transfer_learning_model(input_shape=(100, 100, 3), num_classes=2):
    """
    VGG16 기반 Transfer Learning 모델
    """
    # VGG16 사전 학습 모델 로드 (ImageNet 가중치)
    base_model = VGG16(
        weights='imagenet',
        include_top=False,  # 최상위 분류 레이어 제외
        input_shape=input_shape
    )
    
    # 사전 학습 레이어 동결 (학습하지 않음)
    for layer in base_model.layers:
        layer.trainable = False
    
    # 새로운 분류 레이어 추가
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.5)(x)
    predictions = Dense(num_classes, activation='softmax')(x)
    
    # 전체 모델 구성
    model = Model(inputs=base_model.input, outputs=predictions)
    
    # 컴파일
    model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model

# Transfer Learning 모델 생성 (선택사항)
# transfer_model = create_transfer_learning_model(input_shape=(100, 100, 3), num_classes=2)
# transfer_model.summary()

print("Transfer Learning 모델 정의 완료! (필요 시 주석 해제하여 사용)")